<a href="https://colab.research.google.com/github/jlab-sensing/MFC_Modeling/blob/main/dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade hepml
# reload modules before executing user code
%load_ext autoreload
# reload all modules every time before executing Python code
%autoreload 2
# render plots in notebook
%matplotlib inline
import datetime
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from hepml.core import plot_regression_tree
sns.set(color_codes=True)
sns.set_palette(sns.color_palette("muted"))
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error as MAPE

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!unzip drive/MyDrive/"jLab Shared Docs"/"MFC Modeling"/stanfordMFCDataset

In [ ]:
#Load teros data
teros_files = glob.glob("rocket4/TEROSoutput*.csv")
X = pd.DataFrame()
for f in teros_files:
  try:
    csv = pd.read_csv(f, index_col=False).dropna()
    X = pd.concat([X, csv])
  except:
    continue

In [ ]:
#Load power data
power_files = glob.glob("rocket4/soil*.csv")
y = pd.DataFrame()
for f in power_files:
  try:
    csv = pd.read_csv(f, on_bad_lines='skip', skiprows=10).dropna(how='all')
    csv = csv.rename({'Unnamed: 0': 'timestamp'}, axis='columns')
    y = pd.concat([y,csv])
  except:
    continue
y["timestamp"] = y["timestamp"].round(decimals = 1)

In [ ]:
#Sort data by timestamp, convert to datetime
X = X.sort_values(['timestamp'])
y = y.sort_values(['timestamp'])
X['timestamp'] = pd.to_datetime(X['timestamp'], unit='s')
y['timestamp'] = pd.to_datetime(y['timestamp'], unit='s')

#Merge data by timestamp
uncut_df = pd.merge_asof(left=X,right=y,direction='nearest',tolerance=pd.Timedelta('0.1 min'), on = 'timestamp').dropna(how='all')

#Isolate data from cell0
df = uncut_df.loc[uncut_df['sensorID'] == 0]

#Use only data after deployment data
df = df.loc[df['timestamp'] > '2021-06-11']

#Calculate power
df["power"] = np.abs(np.multiply(df.iloc[:, 8]*10E-12, df.iloc[:, 9]*10E-9))


#Add power time series
df['previous_power - 1'] = df['power'].shift(1).dropna()
#df['previous_power - 2'] = df['power'].shift(2).dropna()
#df['previous_power - 3'] = df['power'].shift(3).dropna()
#df['previous_power - 4'] = df['power'].shift(4).dropna()

df = df.dropna()

In [ ]:
#Data visualization
#!/usr/bin/env python3
!pip install arrow
import matplotlib as mpl
mpl.use('Agg')
#mpl.rc('font', **font)
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.dates as md
import datetime
import numpy as np
from pytz import timezone
import pandas as pd
from glob import glob
import arrow
import os
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

# Limits for graphs
VOLTAGE_LIM = 1
CURRENT_LIM = 200
POWER_LIM = 50

mv = df.rolling(5*60).mean()

plt.close()
plt.xlabel("Time")
fig, (ax1, ax3) = plt.subplots(2,figsize=(4,2), sharex=True)
fig.autofmt_xdate()


volt_color= 'tab:blue'

amp_color = 'tab:red'


volt_color1= 'tab:blue'
volt_style1 = 'dashed'
volt_color2= 'tab:green'
volt_style2 = 'dotted'
amp_color1 = 'tab:red'
amp_style1='dashed'
amp_color2 = 'tab:orange'
amp_style2='dashdot'
ax1.set_ylabel('Cell Voltage (V)')
ax1.plot(df['timestamp'], mv['V1 [10nV]'], color=volt_color1, ls=volt_style1)
#ax1.plot(df['timestamp'], mv['V2 [10nV]'], color=volt_color2, ls=volt_style2)
ax1.tick_params(axis='y', labelcolor=volt_color1)
ax1.set_ylim(0, VOLTAGE_LIM)

ax2 = ax1.twinx()
ax2.set_ylabel('Current (μA)')
ax2.plot(df['timestamp'], 1E6*mv['I1L [10pA]'], color=amp_color1, ls=amp_style1)
#ax2.plot(df['timestamp'], 1E6*mv['I2L [10pA]'], color=amp_color2, ls=amp_style2)
ax2.tick_params(axis='y', labelcolor=amp_color1)
ax2.set_ylim(0, CURRENT_LIM)
ax1.tick_params(axis='x', which='both', length=0)
ax2.tick_params(axis='x', which='both', length=0)

ax1.grid(True)
ax1.legend(['C1 volts','C2 volts'], loc='upper left', prop={'size': 6})
#ax2.legend(['C1 amps','C2 amps'], loc='upper right' , prop={'size': 6})
# Re-arrange legends, ensures data does not draw on top of them
all_axes = fig.get_axes()
for axis in all_axes:
    legend = axis.get_legend()
    if legend is not None:
        legend.remove()
        all_axes[-1].add_artist(legend)

ax3.fmt_xdata = md.DateFormatter('%m-%d-%y')
ax3.xaxis.set_major_formatter(md.DateFormatter('%m-%d-%y'))
ax3.set_ylabel("Power (uW)")
ax3.grid(True)
#print('max power: ',max(max(1E6*df['power1']),max(1E6*df['power2'])))
ax3.set_ylim(0, 5)
ax3.plot(df['timestamp'], 1E6*mv['power'], color=volt_color1, ls = volt_style1)
#ax3.plot(df['timestamp'], 1E6*mv['power2'], color=volt_color2, ls = volt_style2)
ax3.legend(['Cell 1','Cell 2'], loc='upper right', prop={'size': 6})
#ax3.legend(['Cell 1','Cell 2'], loc='upper left', prop={'size': 6})
ax3.tick_params(axis='x', labelsize=6, rotation=30)
ax3.xaxis.set_major_locator(plt.MaxNLocator(8))
#ax3.set_xlim(mv.index[0], datetime.date(2020,5,19))
for label in ax3.get_xticklabels():
    label.set_horizontalalignment('center')

plt.tight_layout(pad=0.6, w_pad=0.5, h_pad=0.6)
plt.subplots_adjust(hspace=0.15)
plt.savefig('twobat.pdf')
plt.close()
#tot_energy = np.trapz(df['power1'])
#tot_energy = np.trapz(df['power2'])
#print(tot_energy)
#print((df.tail(1).index - df.head(1).index).total_seconds())


In [ ]:
#Re-split data for training
X = pd.concat([df.iloc[:, 0:1], df.iloc[:, 2:5], df.iloc[:, 14:18]], axis = 1).dropna()
y = df.iloc[:, 13:14].dropna()

#Convert datetime to timestamp for training
X["timestamp"] = X["timestamp"].values.astype("float64")

#Creating training and testing sets
X_train, X_test = train_test_split(X, test_size=0.3, shuffle=False)
y_train, y_test = train_test_split(y, test_size=0.3, shuffle=False)

In [ ]:
df[pd.isnull(df).any(axis=1)]

,timestamp,sensorID,raw_VWC,temp,EC,I1L_valid,I2L_valid,I1H [nA],I1L [10pA],V1 [10nV],V2 [10nV],I2H [nA],I2L [10pA],power,previous_power - 1
840340,2021-06-21 17:17:22,0.0,2468.52,32.5,219.0,1.0,1.0,-41844.0,-724790.0,-1658645.0,-5328466.0,130811.0,1735386.0,1.202169e-07,NaN


In [ ]:
#Train model
model = RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=42)
%time model.fit(X_train, y_train.values.ravel())

CPU times: user 1min 11s, sys: 0 ns, total: 1min 11s
Wall time: 44 s


RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=42)

In [ ]:
#Define SMAPE
def SMAPE(actual, predicted):
    return 1/len(actual) * np.sum(2 * np.abs(predicted-actual) / (np.abs(actual) + np.abs(predicted))*100)

In [ ]:
#Evaluate SMAPE
print("Train SMAPE:\n", SMAPE(y_train.values.ravel(), model.predict(X_train)))
print("Test SMAPE:\n", SMAPE(y_test.values.ravel(), model.predict(X_test)))


Train SMAPE:
 42.68911877066924
Test SMAPE:
 105.63993642370252


In [ ]:
df["power pred"] = model.predict(X)